In [1]:
# fine-tune scibert on Zhao et al. dataset and make inference on our own
import pandas as pd
import torch
import numpy as np

ImportError: No module named 'torch'

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
artifact_type_map = {'Material':0,'Method':1,'Supplement':2}

In [4]:
type_artifact_map = {0:'Material',1:'Method',2:'Supplement'}

In [5]:
# available from Zhao et al. https://aclanthology.org/D19-1524/, https://github.com/zhaohe1995/SciRes
# first load the dataset
f = open('role_1st_train.txt')
line = f.readline()
Context = []
Label = []
while line:
    Context.append(line.split('__label__')[0])
    Label.append(artifact_type_map[line.split('__label__')[1].split('|')[0]])
    line = f.readline()
    
df_train = pd.DataFrame({'label':Label,'text':Context})

# first load the dataset
f = open('test.txt')
line = f.readline()
Context = []
Label = []
while line:
    Context.append(line.split('__label__')[0])
    Label.append(artifact_type_map[line.split('__label__')[1].split('|')[0]])
    line = f.readline()
    
df_test = pd.DataFrame({'label':Label,'text':Context})

from datasets import Dataset
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

In [6]:
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/hanchengc/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file h

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=256)

In [8]:
tokenized_dataset_train = dataset_train.map(preprocess_function, batched=True)
tokenized_dataset_test = dataset_test.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=3).to("cuda")

loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home/hanchengc/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,

In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.05,
)

trainer = Trainer(
    model=model,
    args=training_args,
    #train_dataset=tokenized_imdb["train"],
    train_dataset=tokenized_dataset_train,
    #eval_dataset=tokenized_imdb["test"],
    
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2988
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size

Step,Training Loss
500,0.442000


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=935, training_loss=0.3136039937881225, metrics={'train_runtime': 267.1611, 'train_samples_per_second': 55.921, 'train_steps_per_second': 3.5, 'total_flos': 1965457230428160.0, 'train_loss': 0.3136039937881225, 'epoch': 5.0})

In [12]:
trainer.save_model("scibert_artifact_classifier")

Saving model checkpoint to scibert_artifact_classifier
Configuration saved in scibert_artifact_classifier/config.json
Model weights saved in scibert_artifact_classifier/pytorch_model.bin
tokenizer config file saved in scibert_artifact_classifier/tokenizer_config.json
Special tokens file saved in scibert_artifact_classifier/special_tokens_map.json


In [2]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

ImportError: No module named 'datasets'

In [14]:
predictions = trainer.predict(tokenized_dataset_test)
print(predictions.predictions.shape, predictions.label_ids.shape)

import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

import evaluate

metric = evaluate.load("f1")
metric.compute(predictions=preds, references=predictions.label_ids, average='micro')

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 333
  Batch size = 16


(333, 3) (333,)


{'f1': 0.8258258258258259}

In [15]:
metric = evaluate.load("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8258258258258259}

In [40]:
preds

array([0, 1, 1, 2, 1, 1, 2, 1, 0, 1, 0, 0, 2, 0, 0, 1, 2, 0, 2, 1, 2, 1,
       1, 0, 0, 0, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 2, 0,
       0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 2,
       2, 1, 2, 1, 0, 2, 2, 0, 2, 1, 2, 1, 1, 0, 2, 0, 2, 2, 2, 1, 1, 0,
       1, 0, 1, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 2, 0,
       2, 2, 1, 0, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 0, 1, 1, 2, 2, 2, 0, 1,
       1, 2, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2,
       1, 2, 1, 1, 2, 0, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 0, 0, 0, 2, 1, 0,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2,
       2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 1, 2, 1, 0, 1,
       0, 2, 2, 1, 0, 2, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 0, 0, 0, 2, 2, 1,
       1, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 2, 1,
       1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2,
       2, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,

In [41]:
predictions.label_ids

array([0, 1, 1, 2, 1, 1, 2, 2, 0, 1, 0, 2, 2, 0, 0, 1, 2, 0, 2, 2, 1, 1,
       1, 0, 1, 0, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 1,
       0, 0, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 0, 0, 2, 0, 1,
       2, 1, 2, 1, 0, 2, 2, 0, 2, 2, 0, 1, 1, 0, 2, 0, 2, 2, 2, 1, 1, 0,
       1, 0, 1, 2, 0, 2, 1, 2, 2, 2, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 2, 1,
       0, 2, 1, 0, 2, 2, 2, 1, 2, 1, 1, 2, 0, 1, 2, 1, 0, 2, 2, 1, 0, 1,
       1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 0, 0, 2, 0, 1, 0, 1, 0, 2, 0, 0,
       1, 2, 1, 1, 2, 0, 1, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 0, 2, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 2, 2, 2, 0, 0, 2, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 2, 1, 0, 2, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 2, 2, 1,
       1, 1, 2, 0, 2, 1, 0, 1, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 2, 2, 1,
       1, 2, 1, 1, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2,
       2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1,

In [5]:
temp1 = [0, 1, 1, 2, 1, 1, 2, 1, 0, 1, 0, 0, 2, 0, 0, 1, 2, 0, 2, 1, 2, 1,
       1, 0, 0, 0, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 2, 0,
       0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 2,
       2, 1, 2, 1, 0, 2, 2, 0, 2, 1, 2, 1, 1, 0, 2, 0, 2, 2, 2, 1, 1, 0,
       1, 0, 1, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 2, 0,
       2, 2, 1, 0, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 0, 1, 1, 2, 2, 2, 0, 1,
       1, 2, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2,
       1, 2, 1, 1, 2, 0, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 0, 0, 0, 2, 1, 0,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2,
       2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 1, 2, 1, 0, 1,
       0, 2, 2, 1, 0, 2, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 0, 0, 0, 2, 2, 1,
       1, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 2, 1,
       1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2,
       2, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 0, 2, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 2, 2, 2, 0, 0,
       2, 0, 0]

temp2 = [0, 1, 1, 2, 1, 1, 2, 2, 0, 1, 0, 2, 2, 0, 0, 1, 2, 0, 2, 2, 1, 1,
       1, 0, 1, 0, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 1,
       0, 0, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 0, 0, 2, 0, 1,
       2, 1, 2, 1, 0, 2, 2, 0, 2, 2, 0, 1, 1, 0, 2, 0, 2, 2, 2, 1, 1, 0,
       1, 0, 1, 2, 0, 2, 1, 2, 2, 2, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 2, 1,
       0, 2, 1, 0, 2, 2, 2, 1, 2, 1, 1, 2, 0, 1, 2, 1, 0, 2, 2, 1, 0, 1,
       1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 0, 0, 2, 0, 1, 0, 1, 0, 2, 0, 0,
       1, 2, 1, 1, 2, 0, 1, 2, 2, 1, 2, 0, 2, 1, 2, 1, 0, 0, 0, 2, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 2, 2, 2, 0, 0, 2, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 2, 1, 0, 2, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 2, 2, 1,
       1, 1, 2, 0, 2, 1, 0, 1, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 2, 2, 1,
       1, 2, 1, 1, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2,
       2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 1, 2, 1,
       0, 2, 1, 1, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 0, 0, 1, 2, 2, 2, 2, 2,
       2, 0, 0]

In [6]:
import sklearn.metrics 

In [8]:
sklearn.metrics.f1_score(temp1, temp2, average='macro')

0.8308130439720092

In [10]:
sklearn.metrics.accuracy_score(temp1, temp2)

0.8318318318318318

In [12]:
sklearn.metrics.precision_score(temp1, temp2, average ='macro')

0.8312661755312561

In [13]:
sklearn.metrics.recall_score(temp1, temp2, average ='macro')

0.8308109671154563

In [16]:
# results on our labelled validation set
df_validation = pd.read_csv('artifact_pred_validation.csv')

In [20]:

sklearn.metrics.f1_score(df_validation['label'],df_validation['label_actual'],average='macro')

0.9441045441045443

In [21]:

sklearn.metrics.accuracy_score(df_validation['label'],df_validation['label_actual'])

0.95

In [22]:

sklearn.metrics.precision_score(df_validation['label'],df_validation['label_actual'],average='macro')

0.9298245614035087

In [23]:

sklearn.metrics.recall_score(df_validation['label'],df_validation['label_actual'],average='macro')

0.9652777777777778

# predict on our own dataset

In [16]:

model_new = AutoModelForSequenceClassification.from_pretrained("scibert_artifact_classifier", num_labels=3).to("cuda")

loading configuration file scibert_artifact_classifier/config.json
Model config BertConfig {
  "_name_or_path": "scibert_artifact_classifier",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading weights file scibert_artifact_classifier/pyt

In [17]:
predictions = trainer.predict(tokenized_dataset_test)
print(predictions.predictions.shape, predictions.label_ids.shape)

import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

import evaluate

metric = evaluate.load("f1")
metric.compute(predictions=preds, references=predictions.label_ids, average='macro')

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 333
  Batch size = 16


(333, 3) (333,)


{'f1': 0.8252342665905821}

In [88]:
metric = evaluate.load("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8318318318318318}

In [89]:
preds

array([0, 1, 1, 2, 1, 1, 2, 1, 0, 1, 0, 0, 2, 0, 0, 1, 2, 0, 2, 1, 2, 1,
       1, 0, 0, 0, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 2, 0,
       0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 2,
       2, 1, 2, 1, 0, 2, 2, 0, 2, 1, 2, 1, 1, 0, 2, 0, 2, 2, 2, 1, 1, 0,
       1, 0, 1, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 2, 0,
       2, 2, 1, 0, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 0, 1, 1, 2, 2, 2, 0, 1,
       1, 2, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2,
       1, 2, 1, 1, 2, 0, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 0, 0, 0, 2, 1, 0,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2,
       2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 1, 2, 1, 0, 1,
       0, 2, 2, 1, 0, 2, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 0, 0, 0, 2, 2, 1,
       1, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 2, 1,
       1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2,
       2, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,

In [18]:
df = pd.read_csv('../../extracted_url_data_arxiv_normalized_field.tsv',sep='\t')

/tmp/ipykernel_30/2959758657.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/net/s3/s2-research/hanchengc/cleaned_arxiv_url_context/extracted_url_data_arxiv_normalized_field.tsv',sep='\t')


In [19]:
df

,PAPER_ID,SEC_INDEX,POS_INDEX,SECTION,IsFOOTNOTE,FOOTNOTE,CONTEXT,CONTEXT_PAR,URLS,subdomain,...,suffix,URLS_normalized,year,URLS_reextracted,URLS_reextracted_normalized,URLS_reextracted_normalized_stripped,year_range,categories,categories_top,categories_top_final
0,astroph0001519,body_text,1,Introduction,0,NaN,Early photometric observations consist of main...,Early photometric observations consist of main...,http://www.kusastro.kyoto-u.ac.jp/vsnet,www.kusastro,...,ac.jp,http://www.kusastro.kyoto-u.ac.jp/vsnet,2000,http://www.kusastro.kyoto-u.ac.jp/vsnet,http://www.kusastro.kyoto-u.ac.jp/vsnet,kusastro.kyoto-u.ac.jp/vsnet,1996-2000,astro-ph,astro-ph,physics
1,astroph0001298,body_text,78,Discussion,0,NaN,It is interesting to note that the capabilitie...,One way around the uncertainties in assuming e...,http://maxim.gsfc.nasa.gov,maxim.gsfc,...,gov,http://maxim.gsfc.nasa.gov/,2000,http://maxim.gsfc.nasa.gov,http://maxim.gsfc.nasa.gov/,maxim.gsfc.nasa.gov/,1996-2000,astro-ph,astro-ph,physics
2,astroph0001299,body_text,5,Search and Analysis,1,Via: http://heasarc.gsfc.nasa.gov/W3Browse,We have searched the ROSAT public database FOO...,We have searched the ROSAT public database FOO...,http://heasarc.gsfc.nasa.gov/W3Browse,heasarc.gsfc,...,gov,http://heasarc.gsfc.nasa.gov/W3Browse,2000,http://heasarc.gsfc.nasa.gov/W3Browse,http://heasarc.gsfc.nasa.gov/W3Browse,heasarc.gsfc.nasa.gov/W3Browse,1996-2000,astro-ph,astro-ph,physics
3,astroph0001006,body_text,10,Two-point correlation function,1,for more details see http://www.batse.msfc.nas...,Another major source of uncertainty comes from...,2. From Figure 1 (Left Panel) it is seen that ...,http://www.batse.msfc.nasa.gov/batse/grb/catalog,www.batse.msfc,...,gov,http://www.batse.msfc.nasa.gov/batse/grb/catalog,2000,http://www.batse.msfc.nasa.gov/batse/grb/catalog,http://www.batse.msfc.nasa.gov/batse/grb/catalog,batse.msfc.nasa.gov/batse/grb/catalog,1996-2000,astro-ph,astro-ph,physics
4,astroph0001006,body_text,23,Conclusions and Summary,0,NaN,"Greiner, J. , http://www.aip.de/People/JGreine...","Greiner, J. , http://www.aip.de/People/JGreine...",http://www.aip.de/People/JGreiner/grbgen.html,www,...,de,http://www.aip.de/People/JGreiner/grbgen.html,2000,http://www.aip.de/People/JGreiner/grbgen.html,http://www.aip.de/People/JGreiner/grbgen.html,aip.de/People/JGreiner/grbgen.html,1996-2000,astro-ph,astro-ph,physics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805701,nuclth9907096,body_text,235,Total pion multiplicity::Bose-Einstein distrib...,0,NaN,"S. Schönfelder, PhD thesis, MPI für Physik, Mü...","S. Schönfelder, PhD thesis, MPI für Physik, Mü...",http://na49info.cern.ch/cgi-bin/wwwd-util,na49info,...,ch,http://na49info.cern.ch/cgi-bin/wwwd-util,1999,http://na49info.cern.ch/cgi-bin/wwwd-util,http://na49info.cern.ch/cgi-bin/wwwd-util,na49info.cern.ch/cgi-bin/wwwd-util,1996-2000,nucl-th,nucl-th,physics
805702,nuclth9907096,body_text,237,Total pion multiplicity::Bose-Einstein distrib...,0,NaN,"H. Appelshäuser, PhD thesis, Johann Wolfgang G...","H. Appelshäuser, PhD thesis, Johann Wolfgang G...",http://na49info.cern.ch/cgi-bin/wwwd-util/NA49...,na49info,...,ch,http://na49info.cern.ch/cgi-bin/wwwd-util/NA49...,1999,http://na49info.cern.ch/cgi-bin/wwwd-util/NA49...,http://na49info.cern.ch/cgi-bin/wwwd-util/NA49...,na49info.cern.ch/cgi-bin/wwwd-util/NA49/NOTE?150,1996-2000,nucl-th,nucl-th,physics
805703,nuclth9907096,body_text,247,Total pion multiplicity::Bose-Einstein distrib...,0,NaN,"F. James, MINUIT – Function Minimization and E...","F. James, MINUIT – Function Minimization and E...",http://consult.cern.ch/writeups/minuit,consult,...,ch,http://consult.cern.ch/writeups/minuit,1999,http://consult.cern.ch/writeups/minuit,http://consult.cern.ch/writeups/minuit,consult.cern.ch/writeups/minuit,1996-2000,nucl-th,nucl-th,physics
805704,physics9907048,abstract,7,Abstract,0,NaN,URL: http://ahobon.reduaz.mx/valeri/valeri.htm,URL: http://ahobon.reduaz.mx/valeri/val

In [52]:
df_test

,label,text
0,0,"In this paper , the YAiTRON dictionary is expl..."
1,1,are used with and without 3 leading supervised...
2,1,"2011 ], based on multilayer perceptron neural ..."
3,2,I am grateful to Rodney Douglas and Kevan Mart...
4,1,The difference between reading and dictation t...
...,...,...
328,2,The majority of the English descriptions were ...
329,2,"For instance , a matrix of P - values resultin..."
330,2,Recent studies show that language varieties ca...
331,0,These pairs were classified manually in Cognat...


In [20]:

#df_real = df[0:100]
#df_real = df.sample(100)
df_real = df
df_real['PAPER_ID'] = df_real['PAPER_ID'].astype('str')
df_real['text'] = df['CONTEXT_PAR']
dataset_topredict = Dataset.from_pandas(df_real)
tokenized_dataset_topredict = dataset_topredict.map(preprocess_function, batched=True)

  0%|          | 0/806 [00:00<?, ?ba/s]

In [22]:
predictions_real = trainer.predict(tokenized_dataset_topredict)
preds_real = np.argmax(predictions_real.predictions, axis=-1)
preds_real

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: URLS, SECTION, domain, categories, categories_top, FOOTNOTE, IsFOOTNOTE, URLS_reextracted_normalized, SEC_INDEX, PAPER_ID, CONTEXT_PAR, text, CONTEXT, categories_top_final, suffix, URLS_normalized, POS_INDEX, URLS_reextracted_normalized_stripped, URLS_reextracted, year, year_range, subdomain. If URLS, SECTION, domain, categories, categories_top, FOOTNOTE, IsFOOTNOTE, URLS_reextracted_normalized, SEC_INDEX, PAPER_ID, CONTEXT_PAR, text, CONTEXT, categories_top_final, suffix, URLS_normalized, POS_INDEX, URLS_reextracted_normalized_stripped, URLS_reextracted, year, year_range, subdomain are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 805706
  Batch size = 16


array([2, 1, 0, ..., 1, 2, 2])

In [23]:
df_real['label'] = preds_real
df_real['label'] = df_real['label'].map(type_artifact_map)
df_real

,PAPER_ID,SEC_INDEX,POS_INDEX,SECTION,IsFOOTNOTE,FOOTNOTE,CONTEXT,CONTEXT_PAR,URLS,subdomain,...,year,URLS_reextracted,URLS_reextracted_normalized,URLS_reextracted_normalized_stripped,year_range,categories,categories_top,categories_top_final,text,label
0,astroph0001519,body_text,1,Introduction,0,NaN,Early photometric observations consist of main...,Early photometric observations consist of main...,http://www.kusastro.kyoto-u.ac.jp/vsnet,www.kusastro,...,2000,http://www.kusastro.kyoto-u.ac.jp/vsnet,http://www.kusastro.kyoto-u.ac.jp/vsnet,kusastro.kyoto-u.ac.jp/vsnet,1996-2000,astro-ph,astro-ph,physics,Early photometric observations consist of main...,Supplement
1,astroph0001298,body_text,78,Discussion,0,NaN,It is interesting to note that the capabilitie...,One way around the uncertainties in assuming e...,http://maxim.gsfc.nasa.gov,maxim.gsfc,...,2000,http://maxim.gsfc.nasa.gov,http://maxim.gsfc.nasa.gov/,maxim.gsfc.nasa.gov/,1996-2000,astro-ph,astro-ph,physics,One way around the uncertainties in assuming e...,Method
2,astroph0001299,body_text,5,Search and Analysis,1,Via: http://heasarc.gsfc.nasa.gov/W3Browse,We have searched the ROSAT public database FOO...,We have searched the ROSAT public database FOO...,http://heasarc.gsfc.nasa.gov/W3Browse,heasarc.gsfc,...,2000,http://heasarc.gsfc.nasa.gov/W3Browse,http://heasarc.gsfc.nasa.gov/W3Browse,heasarc.gsfc.nasa.gov/W3Browse,1996-2000,astro-ph,astro-ph,physics,We have searched the ROSAT public database FOO...,Material
3,astroph0001006,body_text,10,Two-point correlation function,1,for more details see http://www.batse.msfc.nas...,Another major source of uncertainty comes from...,2. From Figure 1 (Left Panel) it is seen that ...,http://www.batse.msfc.nasa.gov/batse/grb/catalog,www.batse.msfc,...,2000,http://www.batse.msfc.nasa.gov/batse/grb/catalog,http://www.batse.msfc.nasa.gov/batse/grb/catalog,batse.msfc.nasa.gov/batse/grb/catalog,1996-2000,astro-ph,astro-ph,physics,2. From Figure 1 (Left Panel) it is seen that ...,Supplement
4,astroph0001006,body_text,23,Conclusions and Summary,0,NaN,"Greiner, J. , http://www.aip.de/People/JGreine...","Greiner, J. , http://www.aip.de/People/JGreine...",http://www.aip.de/People/JGreiner/grbgen.html,www,...,2000,http://www.aip.de/People/JGreiner/grbgen.html,http://www.aip.de/People/JGreiner/grbgen.html,aip.de/People/JGreiner/grbgen.html,1996-2000,astro-ph,astro-ph,physics,"Greiner, J. , http://www.aip.de/People/JGreine...",Supplement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805701,nuclth9907096,body_text,235,Total pion multiplicity::Bose-Einstein distrib...,0,NaN,"S. Schönfelder, PhD thesis, MPI für Physik, Mü...","S. Schönfelder, PhD thesis, MPI für Physik, Mü...",http://na49info.cern.ch/cgi-bin/wwwd-util,na49info,...,1999,http://na49info.cern.ch/cgi-bin/wwwd-util,http://na49info.cern.ch/cgi-bin/wwwd-util,na49info.cern.ch/cgi-bin/wwwd-util,1996-2000,nucl-th,nucl-th,physics,"S. Schönfelder, PhD thesis, MPI für Physik, Mü...",Method
805702,nuclth9907096,body_text,237,Total pion multiplicity::Bose-Einstein distrib...,0,NaN,"H. Appelshäuser, PhD thesis, Johann Wolfgang G...","H. Appelshäuser, PhD thesis, Johann Wolfgang G...",http://na49info.cern.ch/cgi-bin/wwwd-util/NA49...,na49info,...,1999,http://na49info.cern.ch/cgi-bin/wwwd-util/NA49...,http://na49info.cern.ch/cgi-bin/wwwd-util/NA49...,na49info.cern.ch/cgi-bin/wwwd-util/NA49/NOTE?150,1996-2000,nucl-th,nucl-th,physics,"H. Appelshäuser, PhD thesis, Johann Wolfgang G...",Supplement
805703,nuclth9907096,body_text,247,Total pion multiplicity::Bose-Einstein distrib...,0,NaN,"F. James, MINUIT – Function Minimization and E...","F. James, MINUIT – Function Minimization and E...",http://consult.cern.ch/writeups/minuit,consult,...,1999,http://consult.cern.ch/writeups/minuit,http://consult.cern.ch/writeups/minuit,consult.cern.ch/writeups/minuit,1996-2000,nucl-th,nucl-th,physics,"F. James, MINUIT – Function Minimization and E...",Method
805704,physics9907048,abst